In [ ]:
import numpy as np

hexa = ['0', '1', '2', '3', '4', '5', '6', '7', 
        '8', '9', 'A', 'B', 'C', 'D', 'E', 'F']
output = "6E626264000000000188808065E16CE26FEEEBEC65E16CE2018C898865E16CE20004090800000000018C898865E16CE26E666B6C000000006E62626400000000"

rem = 15 #substitute values from 0-15 to get m1, m2 in sequence
b = 1600
l = 512
c = 1024
r = 576
rounds = 24

state = np.zeros((5,5,64))
tempstate = np.zeros((5,5,64))
four_bits = ['0000', '0001', '0010', '0011',
             '0100', '0101', '0110', '0111',
             '1000', '1001', '1010', '1011',
             '1100', '1101', '1110', '1111']

given_str = ''
for i in range(rem,len(output),16):
    given_str += output[i]
print(given_str)

for m2 in four_bits:
    for m1 in four_bits:
        
        message = '0000' * rem + m1 + '0' * (60 - rem*4) + 
        '0000' * rem + m2 + '0' * (60 - rem*4) + '0' * 448

        for i in range(5):
            for j in range(5):
                for k in range(64):
                    state[i,j,k] = 0

        for k in range(r):
            state[k//(64*5), (k//64)%5, k%64] = message[k]


        current_round = 0
        column_parity = np.zeros((5,64))


        while (current_round < rounds):

            #theta operation
            for i in range(5):
                for k in range(64):
                    column_parity[i, k] = 0
                    for j in range(5):
                        column_parity[i, k] = int(column_parity[i, k]) ^ int(state[i, j, k])

            for i in range(5):
                for j in range(5):
                    for k in range(64):
                        state[i, j, k] = int(state[i, j, k]) ^ (int(column_parity[(i+4)%5, k]) ^ int(column_parity[(i+1)%5, k]))
                        tempstate[i, j, k] = state[i, j, k]


            #pi operation
            for i in range(5):
                for j in range(5):
                    for k in range(64):
                        state[j, ((2 * i) + (3 * j)) % 5, k] = tempstate[i, j, k]


            #chi operation
            for i in range(5):
                for j in range(5):
                    for k in range(64):
                        tempstate[i, j, k] = state[i, j, k]

            for i in range(5):
                for j in range(5):
                    for k in range(64):
                        state[i, j, k] = int(tempstate[i, j, k]) ^ int(~int(tempstate[i, (j+1)%5, k]) & int(tempstate[i, (j+2)%5, k]))

            current_round += 1


        k = 0
        out_str = ''
        while (k < l):

            index = 0
            for j in [3,2,1,0]:
                index = index * 2 + state[k//(64*5), (k//64) % 5, (k % 64 + j)]
            if k%64 == 4*rem:
                out_str += hexa[int(index)]
            k += 4
        
        if out_str == given_str:
            print(out_str)
            print(m1)
            print(m2)
            break
    else:
        continue
    break